In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Download the file to the Colab VM
spark.sparkContext.addFile(url)

# SparkFiles.get grabs the local path to the downloaded file
file_path = SparkFiles.get("home_sales_revised.csv")

# Read into a DataFrame
df = (spark.read
            .option("header", True)
            .option("inferSchema", True)
            .csv(file_path))

df.show(5)
df.printSchema()# 2. Create a temporary view of the DataFrame.

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_1 = """
SELECT YEAR(date) AS sale_year,
       ROUND(AVG(price), 2) AS avg_price
FROM   home_sales
WHERE  bedrooms = 4
GROUP  BY sale_year
ORDER  BY sale_year
"""
spark.sql(query_1).show()


+---------+---------+
|sale_year|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built AS year_built,
       ROUND(AVG(price), 2) AS avg_price
FROM   home_sales
WHERE  bedrooms = 3
  AND  bathrooms = 3
GROUP  BY year_built
ORDER  BY year_built
"""
spark.sql(query_2).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [11]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3 = """
SELECT date_built AS year_built,
       ROUND(AVG(price), 2) AS avg_price
FROM   home_sales
WHERE  bedrooms = 3
  AND  bathrooms = 3
  AND  floors    = 2
  AND  sqft_living >= 2000
GROUP  BY year_built
ORDER  BY year_built
"""
spark.sql(query_3).show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
t0 = time.time()

query_4 = """
SELECT view,
       ROUND(AVG(price), 2) AS avg_price
FROM   home_sales
GROUP  BY view
HAVING AVG(price) >= 350000
ORDER  BY view
"""
result_uncached = spark.sql(query_4)
result_uncached.show()

print(f"Runtime (uncached): {time.time() - t0:.3f} seconds")


+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

Runtime (uncached): 0.699 seconds


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
print("Is cached?", spark.catalog.isCached("home_sales"))

Is cached? True


In [15]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
t0 = time.time()
result_cached = spark.sql(query_4)
result_cached.show()
print(f"Runtime (cached): {time.time() - t0:.3f} seconds")

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

Runtime (cached): 0.499 seconds


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
parquet_path = "/content/home_sales_partitioned"   # local path in Colab

(df.write
   .mode("overwrite")
   .partitionBy("date_built")
   .parquet(parquet_path))

In [17]:
# 11. Read the parquet formatted data.
df_parq = spark.read.parquet(parquet_path)
df_parq.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [18]:
# 12. Create a temporary table for the parquet data.
df_parq.createOrReplaceTempView("home_sales_parq")

In [19]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
t0 = time.time()
result_parq = spark.sql(query_4.replace("home_sales", "home_sales_parq"))
result_parq.show()
print(f"Runtime (parquet): {time.time() - t0:.3f} seconds")

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

Runtime (parquet): 0.618 seconds


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
print("Is cached now?", spark.catalog.isCached("home_sales"))

Is cached now? False
